In [2]:
import os
import sys
import pandas as pd
import numpy as np
from preprocess import clean_abstract, preprocess_record

In [ ]:
# 1) Cargar datos
df = pd.read_excel("data_concatenada.xlsx",
                   usecols=["Código VRID", "Título", "Resumen", "Keywords", "Interdisciplinario", "Transdisciplinario"]) \
       .fillna("")
cols = ["Título", "Resumen", "Keywords"]
df[cols] = df[cols].applymap(lambda x: "" if str(x).strip().upper() == "DESCONOCIDO" else str(x).strip())
df["text_for_embedding"] = df.apply(
    lambda r: preprocess_record(r["Título"], r["Resumen"], r["Keywords"]),
    axis=1
)
#df.to_excel("vis_exequiel.xlsx", index=False)

C:\Users\Alonso\AppData\Local\Temp\ipykernel_20108\1084149492.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: "" if str(x).strip().upper() == "DESCONOCIDO" else str(x).strip())
